## <strong> 12. 결측 데이터 처리

In [ ]:
# 필요한 라이브러리
import pandas as pd
import numpy as np

#### <strong> 결측치의 표현: None 객체

In [ ]:
# None 객체를 포함한 배열
arr1 = np.array([1, 2, None, 4])
print(arr1.dtype)

# 결측치가 없는 배열
arr2 = np.array([1, 2, 3, 4])
print(arr2.dtype)

In [ ]:
# Object 객체는 numpy, pandas 연산과 호환되지 않음
arr1.sum()

#### <strong> 결측치의 표현: NaN 값

In [ ]:
# NaN 값을 포함한 배열 객체
arr3 = np.array([1, np.nan, 3, 4])
arr3.dtype

In [ ]:
# NaN 값에 대한 산술 연산
print(1 + np.nan)
print(0 * np.nan)
print(arr3.sum())

In [ ]:
# NaN값을 무시하는 집계함수(numpy)
print(np.nansum(arr3))
print(np.nanmax(arr3))
print(np.nanmin(arr3))

In [ ]:
# Pandas는 None 객체를 자동으로 NaN값으로 변환한다.

ser = pd.Series([1, np.nan, 2, None])
print(ser)

### <strong> Pandas를 이용한 결측치 처리
---

#### <strong>마스킹을 통한 Null 값 표현
+ ```isnull()```
+ ```notnull()```

In [ ]:
# [+] 결측치를 포함한 Series 객체 생성
ser = ...

In [ ]:
# [+] isnull()


In [ ]:
# [+] notnull()


#### <strong> 결측치를 포함한 행 또는 열 삭제
+ dropna()

In [ ]:
# DataFrame 생성 함수
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}

    return pd.DataFrame(data, ind)

df = make_df('ABC', [0, 1, 2])
df

In [ ]:
# [+] NaN 값 추가
...
df

In [ ]:
# [+] 결측치를 포함한 행 삭제
...

In [ ]:
# [+] 결측치를 포함한 열 삭제
...

In [ ]:
# [+] NaN 값 추가
...
df

In [ ]:
# [+] NaN값이 한 개라도 포함되면 행 또는 열 삭제
...

In [ ]:
# [+] 모든 값이 전부 NaN값인 행 또는 열 삭제
...

In [ ]:
# [+] Non-null(정상으로 측정된) 값의 개수가 임계치보다 작은 행을 삭제
...

In [ ]:
# [+] Non-null 값의 개수가 임계치보다 작은 열을 삭제
...

#### <strong> NaN값을 다른 값을 대체
+ ```fillna()```

In [ ]:
# Series 객체 생성
ser = pd.Series([1, np.nan, 2, None, 3],
                index=list('abcde'))
ser

In [ ]:
# [+] 결측치를 특정 값을 대체
...

In [ ]:
# [+] Forward-fill 방식(= LOCF)
...

In [ ]:
# [+] Backward-fill 방식(= NOCB)
...

### <strong> 결측치 처리 예제: Air Quality 데이터셋

In [ ]:
from datetime import datetime
from matplotlib import pyplot as plt

"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

In [ ]:
# 데이터 요약
df.info()

In [ ]:
# 시각화 설정 옵션
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

In [ ]:
# 결측 데이터 시각화
import missingno

missingno.matrix(df)

In [ ]:
# 일산화탄소(CO) 시계열 시각화
df['CO(GT)'].plot()

In [ ]:
# 결측치 대치
imp_locf = df['CO(GT)'].copy().ffill() # LOCF
imp_nocb = df['CO(GT)'].copy().bfill() # NOCB
imp_linear = df['CO(GT)'].copy().interpolate() # 선형 보간
imp_mean = df['CO(GT)'].copy().fillna(df['CO(GT)'].mean())  # 평균값 대체

In [ ]:
# K-NN 대치
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)    # default: 2
imp_knn = df.copy().values
imp_knn = imputer.fit_transform(imp_knn)

In [ ]:
# K-NN 대치 결과 변환(ndarray -> DataFrame)
imp_df = pd.DataFrame(imp_knn, index=imp_locf.index, columns=df.columns)

In [ ]:
# 결측치 처리 결과 시각화
plt.plot(df['CO(GT)'], label='actual', zorder=10)
plt.plot(imp_linear, label='linear interpolation', zorder=3)
plt.plot(imp_nocb, label='nocb', zorder=2)
plt.plot(imp_locf, label='locf', zorder=1)
plt.plot(imp_mean, label='mean substitution', zorder=4)
plt.plot(imp_df['CO(GT)'], label='k-nearest neighbor', zorder=5)
plt.legend(loc='best')
plt.show()